In [1]:
from __future__ import print_function

import os
import sys
import csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ROOT import gROOT

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.activations import relu
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K

from tensorflow.keras.datasets import imdb

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print("Packages Loaded")

ImportError: dynamic module does not define module export function (PyInit_libPyROOT)

We will be working with the IMDB data from keras for this lab. The data is already enocded so I wanted to show an example of what text data looks like before it gets encoded. Below is the stanford sentiment treebank data broken up into its data and the sentiment values.

I wanted to use a more complex dataset for this but the time constraints due to COV-19 have made that difficult. 

In [2]:
# Here I import it as a table using pandas and I have to set the column names manually.
data = pd.read_table('dictionary.txt',index_col=False, header=None)
data.columns = ['Phrase|Index']
# Most corpus text data is set up in a way that has delimiters to seperate the data from the indexs. The delimiter for this data is |
data = data['Phrase|Index'].str.split('|', expand=True)
# The code above split the strings into an id index and the actual data.
data = data.rename(columns={0: 'Phrase', 1: 'phrase_ids'})
data

,Phrase,phrase_ids
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936
...,...,...
239227,zoning ordinances to protect your community fr...,220441
239228,zzzzzzzzz,179256
239229,élan,220442
239230,É,220443


In [3]:
# I then do the same steps for the sentiment value labels for this data.
labels = pd.read_table('sentiment_labels.txt')
labels = labels['phrase ids|sentiment values'].str.split('|', expand=True)
labels = labels.rename(columns={0: 'phrase_ids', 1: 'sentiment_values'})
labels

,phrase_ids,sentiment_values
0,0,0.5
1,1,0.5
2,2,0.44444
3,3,0.5
4,4,0.42708
...,...,...
239227,239227,0.36111
239228,239228,0.38889
239229,239229,0.33333
239230,239230,0.88889


In [4]:
# Here I merged the data into one table using a SQL like join command which combines the tables based on the 'phrase_ids' column.
df = data.merge(labels, how='inner', on='phrase_ids')
# I then set the phrase ids as the index for the dataframe and drop the redundant id column.
df.index = df.phrase_ids
df = df.drop('phrase_ids', axis=1)
df

,Phrase,sentiment_values
phrase_ids,,
0,!,0.5
22935,! ',0.52778
18235,! '',0.5
179257,! Alas,0.44444
22936,! Brilliant,0.86111
...,...,...
220441,zoning ordinances to protect your community fr...,0.13889
179256,zzzzzzzzz,0.19444
220442,élan,0.51389


The above process is very common and you should practice it yourself to get used to the process. I recommend (this isn't graded) to do this yourself with the amazon review data here: http://jmcauley.ucsd.edu/data/amazon/

In [5]:
# This is a pretrained embedder called GloVe. here I had to use the 'quoting=csv.QUOTE_NONE' because the text file included quotation marks.
embed = pd.read_table('glove.6B.100d.txt', engine='python', encoding='utf-8', error_bad_lines=False, header=None, quoting=csv.QUOTE_NONE)

In [6]:
# We see here that the data has two components to it. The beginning of every string is the word that got embedded and it is followed by a long list of numbers.
# These numbers are how the machine learning model trains on data. Normally you will automatically create a trainable embedding in your model (like we will today).
# There are times when you will want to use a pretrained embedding like GloVe to speed up computation time. So its good to know how to use them.
# You can find detailed instructions on how to use GloVe and other pre-trained embeddings in the keras and tensorflow docs.
embed

,0
0,the -0.038194 -0.24487 0.72812 -0.39961 0.0831...
1,", -0.10767 0.11053 0.59812 -0.54361 0.67396 0...."
2,. -0.33979 0.20941 0.46348 -0.64792 -0.38377 0...
3,of -0.1529 -0.24279 0.89837 0.16996 0.53516 0....
4,to -0.1897 0.050024 0.19084 -0.049184 -0.08973...
...,...
399995,chanty -0.15577 -0.049188 -0.064377 0.2236 -0....
399996,kronik -0.094426 0.14725 -0.15739 0.071966 -0....
399997,rolonda 0.36088 -0.16919 -0.32704 0.098332 -0....
399998,zsombor -0.10461 -0.5047 -0.49331 0.13516 -0.3...


So now lets get into the data we are working with today. In the last couple of labs we used CNNs and ResNets a lot. This time we are going to compare CNNs with LSTMs for the purpose of classifying text. The data is setup so that a '0 label' is a negative review and a '1 label' is a postive review.

We want to create machine learning models to automatically detect whether or not a review is positive. This has wide applications for both industry and research and has been extensively researched since 2014. 

In [4]:
# Lets load our data. We will limit the number of words to 5,000 as that is how the data is setup.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

# We pad the data because not all sentences in our data are the same length. We want to use a number that is larger than our largest data. Here I will choose 400.
x_train = sequence.pad_sequences(x_train, maxlen=400)
x_test = sequence.pad_sequences(x_test, maxlen=400)

print('Train: x=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: x=%s, y=%s' % (x_test.shape, y_test.shape))

Train: x=(25000, 400), y=(25000,)
Test: x=(25000, 400), y=(25000,)


In [12]:
# Lets start with a very simple 1D CNN model. We will use this as our baseline for everything else in this lab.
model = Sequential()

# This embedding is a trainable parameter. We aren't using GloVE for this model.
model.add(Embedding(5000,50,input_length=400))
model.add(Dropout(0.2))

# There isn't much of a difference with how 1D and 2D CNNs work. They still use filters and scan the data.
# we will use a similar model as our 2D CNN with the adition of an embedding layer at the beginning.
model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())

model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Activation('relu'))

# We will use a sigmoid and a 1 neuron dense output since our data is binary.
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 74s 3ms/sample - loss: 0.6950 - accuracy: 0.6258 - val_loss: 0.6824 - val_accuracy: 0.5037
Epoch 2/10
25000/25000 [==============================] - 76s 3ms/sample - loss: 0.3918 - accuracy: 0.8230 - val_loss: 0.5191 - val_accuracy: 0.8340
Epoch 3/10
25000/25000 [==============================] - 76s 3ms/sample - loss: 0.2914 - accuracy: 0.8788 - val_loss: 0.3857 - val_accuracy: 0.8243
Epoch 4/10
25000/25000 [==============================] - 66s 3ms/sample - loss: 0.2259 - accuracy: 0.9082 - val_loss: 0.2981 - val_accuracy: 0.8724
Epoch 5/10
25000/25000 [==============================] - 76s 3ms/sample - loss: 0.1798 - accuracy: 0.9288 - val_loss: 0.3092 - val_accuracy: 0.8741
Epoch 6/10
25000/25000 [==============================] - 78s 3ms/sample - loss: 0.1439 - accuracy: 0.9432 - val_loss: 0.3533 - val_accuracy: 0.8734
Epoch 7/10
25000/25000 [==============================] 

#### How well is this model doing? Is it overfitting? If so how could you fix this since we are already applying BatchNorm and dropout?

In [19]:
# Try and apply those fixes here. Can you make a baseline that doesn't overfit? What worked best

model = Sequential()

# This embedding is a trainable parameter. We aren't using GloVE for this model.
model.add(Embedding(5000,50,input_length=400))
model.add(Dropout(0.8))

model.add(Conv1D(16,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(Conv1D(16,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())

model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Activation('relu'))


model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.7090 - accuracy: 0.5301 - val_loss: 0.6469 - val_accuracy: 0.6761
Epoch 2/10
25000/25000 [==============================] - 30s 1ms/sample - loss: 0.5566 - accuracy: 0.7194 - val_loss: 0.4559 - val_accuracy: 0.7893
Epoch 3/10
25000/25000 [==============================] - 31s 1ms/sample - loss: 0.4680 - accuracy: 0.7856 - val_loss: 0.4394 - val_accuracy: 0.7947
Epoch 4/10
25000/25000 [==============================] - 31s 1ms/sample - loss: 0.4263 - accuracy: 0.8065 - val_loss: 0.3959 - val_accuracy: 0.8212
Epoch 5/10
25000/25000 [==============================] - 30s 1ms/sample - loss: 0.4036 - accuracy: 0.8180 - val_loss: 0.4050 - val_accuracy: 0.8179
Epoch 6/10
25000/25000 [==============================] - 30s 1ms/sample - loss: 0.3814 - accuracy: 0.8308 - val_loss: 0.3386 - val_accuracy: 0.8623
Epoch 7/10
25000/25000 [==============================] 

In [3]:
# Now we will make the network more complex by adding more filters to the data. How did this affect training?

model = Sequential()

# This embedding is a trainable parameter. We aren't using GloVE for this model.
model.add(Embedding(5000,50,input_length=400))
model.add(Dropout(0.2))

# There isn't much of a difference with how 1D and 2D CNNs work. They still use filters and scan the data.
# we will use a similar model as our 2D CNN with the adition of an embedding layer at the beginning.
model.add(Conv1D(250,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(Conv1D(250,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())

model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Activation('relu'))

# We will use a sigmoid and a 1 neuron dense output since our data is binary.
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 399s 16ms/sample - loss: 0.7905 - accuracy: 0.6594 - val_loss: 0.6765 - val_accuracy: 0.5016
Epoch 2/10
 9600/25000 [==========>...................] - ETA: 3:46 - loss: 0.3816 - accuracy: 0.8326

KeyboardInterrupt: 

In [11]:
# Now lets add more CNN and BatchNorm layers to the network. Did this have the same affect as 2D CNNs from lab 5?

model = Sequential()

model.add(Embedding(5000,50,input_length=400))
model.add(Dropout(0.2))

model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())

model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())


model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Activation('relu'))

# We will use a sigmoid and a 1 neuron dense output since our data is binary.
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_test, y_test))


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 130s 5ms/sample - loss: 0.6532 - accuracy: 0.6542 - val_loss: 0.6820 - val_accuracy: 0.5231
Epoch 2/10
25000/25000 [==============================] - 145s 6ms/sample - loss: 0.3685 - accuracy: 0.8396 - val_loss: 0.7494 - val_accuracy: 0.5241
Epoch 3/10
25000/25000 [==============================] - 126s 5ms/sample - loss: 0.2827 - accuracy: 0.8846 - val_loss: 0.4201 - val_accuracy: 0.8068
Epoch 4/10
25000/25000 [==============================] - 124s 5ms/sample - loss: 0.2311 - accuracy: 0.9104 - val_loss: 0.3226 - val_accuracy: 0.8678
Epoch 5/10
25000/25000 [==============================] - 121s 5ms/sample - loss: 0.1828 - accuracy: 0.9269 - val_loss: 0.3097 - val_accuracy: 0.8790
Epoch 6/10
25000/25000 [==============================] - 121s 5ms/sample - loss: 0.1539 - accuracy: 0.9420 - val_loss: 0.4001 - val_accuracy: 0.8617
Epoch 7/10
25000/25000 [==========================

#### Do 1D CNNs and 2D CNNs behave the same from the changes we are making?

Now lets look at some LSTMs. LSTMs and RNNs in general were the racehorse of deep learning from 2014-2016. Now they have drastically fallen off of favor in the DL community. The questions we want to answer in this lab are: Why do you think this is? Do you think it was a mistake to stray away from RNNs? What changes do you think we could make to make them better or should we just drop them all together?

The resources to learn more about this debate can be found here: https://towardsdatascience.com/the-fall-of-rnn-lstm-2d1594c74ce0

and here: https://towardsdatascience.com/memory-attention-sequences-37456d271992

and here: https://towardsdatascience.com/visual-attention-model-in-deep-learning-708813c2912c

These are optional readings but they serve to give you a firm foundation on the knowledge of current deep learning thought. Feel free to answer the above questions after we train our LSTMs.

If you don't know anything about RNNs read this: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [13]:
# Now we will make our LSTMs. We will use a smaller batch size as they take longer to train.
# We use the same embedding layers as we did for our CNNs.
model = Sequential()
model.add(Embedding(5000,50,input_length=400))

# Here we will add in our LSTM layers. They should be directly after the embedding layer.
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))

# Now we will cast the LSTM output to a dense layer to sort it. If you haven't noticed, thick dense layers at the end of networks are how every model 'collects its thoughts'.
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile('Nadam', 'binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=[x_test, y_test])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 645s 26ms/sample - loss: 0.5076 - accuracy: 0.7422 - val_loss: 0.3736 - val_accuracy: 0.8332
Epoch 2/5
25000/25000 [==============================] - 651s 26ms/sample - loss: 0.3895 - accuracy: 0.8286 - val_loss: 0.3517 - val_accuracy: 0.8426
Epoch 3/5
25000/25000 [==============================] - 673s 27ms/sample - loss: 0.3244 - accuracy: 0.8684 - val_loss: 0.3308 - val_accuracy: 0.8618
Epoch 4/5
25000/25000 [==============================] - 605s 24ms/sample - loss: 0.2752 - accuracy: 0.8954 - val_loss: 0.3608 - val_accuracy: 0.8528
Epoch 5/5
25000/25000 [==============================] - 638s 26ms/sample - loss: 0.2334 - accuracy: 0.9126 - val_loss: 0.3458 - val_accuracy: 0.8701


How does the basic LSTM compare to the 1D CNN? Is it overfitting as much? is it's testing accuracy better? 

In [4]:
# Now lets add another LSTM layer to our model. Did that improve overfitting/accuracy?

model = Sequential()
model.add(Embedding(5000,50,input_length=400))

# Here we will add in our LSTM layers. They should be directly after the embedding layer.
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))



# Now we will cast the LSTM output to a dense layer to sort it. If you haven't noticed, thick dense layers at the end of networks are how every model 'collects its thoughts'.
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile('Nadam', 'binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=[x_test, y_test])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 857s 34ms/sample - loss: 0.5148 - accuracy: 0.7429 - val_loss: 0.3459 - val_accuracy: 0.8498
Epoch 2/5
25000/25000 [==============================] - 851s 34ms/sample - loss: 0.3327 - accuracy: 0.8635 - val_loss: 0.3114 - val_accuracy: 0.8666
Epoch 3/5
25000/25000 [==============================] - 848s 34ms/sample - loss: 0.2757 - accuracy: 0.8922 - val_loss: 0.3084 - val_accuracy: 0.8760
Epoch 4/5
25000/25000 [==============================] - 851s 34ms/sample - loss: 0.2526 - accuracy: 0.9010 - val_loss: 0.3431 - val_accuracy: 0.8614
Epoch 5/5
25000/25000 [==============================] - 844s 34ms/sample - loss: 0.2215 - accuracy: 0.9158 - val_loss: 0.4130 - val_accuracy: 0.8472


In [5]:
# Now lets use larger LSTM layers. What affect did that have? Why do you think that is based off of your knowledge of RNNs.
model = Sequential()
model.add(Embedding(5000,50,input_length=400))

# Here we will add in our LSTM layers. They should be directly after the embedding layer.
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))

# Now we will cast the LSTM output to a dense layer to sort it. If you haven't noticed, thick dense layers at the end of networks are how every model 'collects its thoughts'.
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile('Nadam', 'binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=[x_test, y_test])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 2637s 105ms/sample - loss: 0.6202 - accuracy: 0.6646 - val_loss: 0.5071 - val_accuracy: 0.7749
Epoch 2/5
25000/25000 [==============================] - 2491s 100ms/sample - loss: 0.6119 - accuracy: 0.6644 - val_loss: 0.5388 - val_accuracy: 0.7497
Epoch 3/5
25000/25000 [==============================] - 2455s 98ms/sample - loss: 0.6125 - accuracy: 0.6332 - val_loss: 0.6907 - val_accuracy: 0.5000
Epoch 4/5
25000/25000 [==============================] - 2464s 99ms/sample - loss: 0.6860 - accuracy: 0.5069 - val_loss: 0.6802 - val_accuracy: 0.6789
Epoch 5/5
25000/25000 [==============================] - 2456s 98ms/sample - loss: 0.6814 - accuracy: 0.5236 - val_loss: 0.6973 - val_accuracy: 0.5070


In [5]:
# Now lets add Bi-directional layers to each of our RNNs. These make the model learn the data scanning both forwards and backwards. 
# Here is a detailed description: https://towardsdatascience.com/understanding-bidirectional-rnn-in-pytorch-5bd25a5dd66

# The bidirectional layer is a wrapper, you can apply it like so to each LSTM layer.
model = Sequential()
model.add(Embedding(5000,50,input_length=400))

model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile('Nadam', 'binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=[x_test, y_test])


#How does this affect training/overfitting? 

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 1412s 56ms/sample - loss: 0.5333 - accuracy: 0.7414 - val_loss: 0.4053 - val_accuracy: 0.8234
Epoch 2/5
25000/25000 [==============================] - 1144s 46ms/sample - loss: 0.3469 - accuracy: 0.8562 - val_loss: 0.3402 - val_accuracy: 0.8553
Epoch 3/5
25000/25000 [==============================] - 1148s 46ms/sample - loss: 0.2741 - accuracy: 0.8908 - val_loss: 0.3580 - val_accuracy: 0.8442
Epoch 4/5
25000/25000 [==============================] - 1132s 45ms/sample - loss: 0.2344 - accuracy: 0.9088 - val_loss: 0.3206 - val_accuracy: 0.8736
Epoch 5/5
25000/25000 [==============================] - 1134s 45ms/sample - loss: 0.2008 - accuracy: 0.9218 - val_loss: 0.3255 - val_accuracy: 0.8666


#### Can you think of anyway to prevent overfitting in an LSTM? got down some ideas and feel free to try them. If you get a signifcant result post it to the discussion board for the rest of the class!

Now that we have looked at the classical examples of 1D CNNs and LSTMs, what do you think are the potential tradeoffs between using each one? Which one makes more sense to use and is there a reason to use LSTMs or RNNs in general for sequential data?

If you are feeling brave and have the extra time I encourage you to impliment an attention layer for both the 1D CNN and bi-directional LSTM and see how much Attention helps. You can also use image attention layers to improve 2D CNNs!